In [18]:
from typing import Any, Dict, List

from langchain import LLMMathChain
from langchain.agents import initialize_agent, Tool
from langchain.callbacks import StdOutCallbackHandler
from langchain.callbacks.manager import CallbackManager, CallbackManagerForChainRun
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentType
from langchain.utilities import GoogleSearchAPIWrapper

In [4]:
class CustomStdOutCallbackHandler(StdOutCallbackHandler):
    """Callback Handler that prints to std out."""

    def on_llm_start(
        self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any
    ) -> None:
        """Print out the prompts."""
        newline = "\n"
        print(f'\n\n\033[1m> Prompts: \n{prompts.join(newline)}\033[0m')
    
    def on_chain_start(
        self, serialized: Dict[str, Any], inputs: Dict[str, Any], **kwargs: Any
    ) -> None:
        """Print out that we are entering a chain."""
        class_name = serialized["name"]
        print(inputs)
        print(f"\n\n\033[1m> Entering new {class_name} chain...\033[0m")

class CustomCallbackManager(CallbackManagerForChainRun, CallbackManager):
    """Callback manager that can be used to handle callbacks from langchain."""


In [13]:
def get_secret_identity(superhero_name: str):
    """Useful for getting the secret identity of a superhero"""
    identities = {
        "Spiderman": "Peter Parker",
        "Daredevil": "Matt Murdoch",
        "Wolverine": "James Howlett"
    }
    return identities[superhero_name]


def get_hometown_by_secret_identity(secret_identity: str):
    """Useful for finding the hometown of a secret identity"""
    hometowns = {
        "Peter Parker": "Queens",
        "Matt Murdoch": "Hell's Kitchen",
        "James Howlett": "Alberta"
    }
    return hometowns[secret_identity]

In [19]:
llm = ChatOpenAI(temperature=0)
search = GoogleSearchAPIWrapper()
math_chain = LLMMathChain(llm=llm)

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to look up current events except anything about spiderman"
    ),
    Tool(
        name="Math",
        func=math_chain.run,
        description="useful for doing math"
    ),
    Tool.from_function(get_secret_identity, name="get_secret_identity", description="useful for finding a superhero's secret identity"),
    Tool.from_function(get_hometown_by_secret_identity, name="get_hometown", description="useful for finding a secret identity's hometown")
]
handler = CustomStdOutCallbackHandler()
callback_manager = CallbackManager([handler])
agent_chain = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, callback_manager=callback_manager)

/Users/charlietruong/micromamba/envs/llm/lib/python3.10/site-packages/langchain/chains/llm_math/base.py:50: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [22]:
agent_chain.run("What is the current temperature where spiderman's lives?")

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


{'input': "What is the current temperature where spiderman's lives?"}


> Entering new AgentExecutor chain...
{'input': "What is the current temperature where spiderman's lives?", 'agent_scratchpad': '', 'stop': ['\nObservation:', '\n\tObservation:']}


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. You have access to the following tools:

Search: useful for when you need to look up current events except anything about spiderman
Math: useful for doing math
get_secret_identity: useful for finding a superhero's secret identity
get_hometown: useful for finding a secret identity's hometown

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Math, get_secret_identity, get_hometown]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can 

'The current temperature where Spiderman lives is 72°F.'